# Génération de musique par Intelligence Artificielle pour la pratique sportive 

## Partie 1: Evaluation *"zero-shot"* d'un algorithme existant 
(également appelé *"modèle de fondation pré-entraîné"*)


### Présentation de MusicGen et ressources computationnelles disponibles

[MusicGen](https://audiocraft.metademolab.com/musicgen.html) est un algorithme (aussi appelé *"modèle d'IA"*) *texte -> musique* développé par l'entreprise Meta (Facebook, Instagram, etc...) et sorti en mai 2023.

Nous allons ici télécharger le modèle directement sur l'ordinateur (*"serveur"*) sur lequel se trouve le présent notebook. Nous pourrons ainsi mieux comprendre les ressources computationnelles qu'il requiert.

Pour des raison de performance, nous utiliserons une carte graphique encore appelée [processeur graphique (GPU)](https://fr.wikipedia.org/wiki/Processeur_graphique). Plus précisément, nous utiliserons un [GPU NVIDIA L4](https://www.nvidia.com/fr-fr/data-center/l4/).

La cellule suivante contient une commande permettant de se rendre compte des ressources disponibles sur le GPU (mémoire, etc...) et de son taux d'utilisation.

In [ ]:
%%sh
nvidia-smi

La cellule ci-dessous contient une commande permettant de visualiser les ressources disponible sur le processeur principal (hors GPU), également appelé [CPU](https://fr.wikipedia.org/wiki/Processeur).

In [ ]:
%%sh
htop

**Tâche 1:** Comparez les ressources du présent serveur aux ressources disponibles sur votre ordinateur. Pour se faire, ouvrez le gestionnaire de tâche:
- Sur Windows: [CTRL] + [ALT] + [Suppr]
- Sur MacOS: [CMD] + [ALT] + [Échap]

### Téléchargement du modèle *MusicGen* et évaluation des ressources utilisées

Les 2 cellules ci-dessous chargent quelques modules utilitaires que nous ne détaillerons pas ici.

In [ ]:
%run utilitaires.ipynb

In [ ]:
%load_ext autoreload
%autoreload 2

Dans la cellule ci-dessous, nous téléchargeons puis chargeons en mémoire l'algorithme MusicGen. Cet algorithme est composé de 2 parties: le `processor` qui transforme la donnée d'entrée (texte notamment) en un format expoitable, et le `model` qui génère la musique à proprement parlé.

In [ ]:
processor = AutoProcessor.from_pretrained(
    "facebook/musicgen-medium", 
    cache_dir="./models"
)

model = AutoModelForTextToWaveform.from_pretrained(
    "facebook/musicgen-medium", 
    cache_dir="./models"
)
model.freeze_text_encoder()
model.freeze_audio_encoder()

torch.cuda.empty_cache()

print(model)
print(processor)

Pour le moment, le `model` est chargé sur le CPU et non le GPU. On peut s'en convaincre en exécutant les cellules ci-dessous:

In [ ]:
print(model.device)

In [ ]:
%%sh
nvidia-smi

On peut désormais "transférer" notre modèle IA du CPU vers le GPU:

In [ ]:
model.to("cuda")
print(model.device)

In [ ]:
%%sh
nvidia-smi

Sur CPU, le temps de calcul pour une même génération de musique est plus long que sur GPU:

In [ ]:
# Exécution sur CPU
# ---------------------------
model.to("cpu")
start = time.time()
text_to_music(
    processor,
    model,
    prompts=[
        "baroque music from the 17th century"
    ],
    max_new_tokens=128,
    sampling_rate=32000,
    do_sample=True,
    guidance_scale=10
)
end = time.time()
print("Temps requis pour générer une musique sur CPU (en secondes): ", round(end - start, 3))

# Exécution sur GPU
# ---------------------------
model.to("cuda")
start = time.time()
text_to_music(
    processor,
    model,
    prompts=[
        "baroque music from the 17th century"
    ],
    max_new_tokens=128,
    sampling_rate=32000,
    do_sample=True,
    guidance_scale=10
)
end = time.time()
print("Temps requis pour générer une musique sur GPU (en secondes): ", round(end - start, 3))

### Zero-shot learning & prompt engineering avec MusicGen

Nous allons à présent tester les capacités de MusicGen en mode "zero-shot" c'est-à-dire sans aucun apprentissage supplémentaire.

MusicGen a été entraîné sur des textes en langue anglaise, il faut donc lui parler Anglais!

In [ ]:
torch.cuda.empty_cache()

text_to_music(
    processor,
    model,
    prompts=[
        "relaxing latino music",
        "baroque music from the 17th century"
    ],
    max_new_tokens=256,
    sampling_rate=32000,
    do_sample=True,
    guidance_scale=10
)

**Tâche 2:** Essayez de générer différentes musiques en faisant varier le `prompt`, le `sampling_rate`, la `guidance_scale` et le paramètre `do_sample`. 
Qu'observez-vous ?

La musique générée dépend beaucoup du *"prompt"*. Le [*"prompt engineering"*](https://fr.wikipedia.org/wiki/Ing%C3%A9nierie_de_prompt) consiste à élaborer le prompt le plus "pertinent" en vue de ce que l'on souhaite générer. C'est un processus très empirique et les techniques varient selon le modèle d'IA utilisé car chaque modèle n'a pas appris de la même manière ni sur les mêmes données.

Une fois que l'on a trouvé une bonne structure de prompt, on peut automatiser la génération de prompt comme dans la cellule ci-dessous:

In [ ]:
print(prompt_engineering(sport_session_name="yoga", music_style="latino"))
print(prompt_engineering(sport_session_name="running", music_style="latino"))
print(prompt_engineering(sport_session_name="weight training", music_style="latino"))

In [ ]:
torch.cuda.empty_cache()

text_to_music(
    processor,
    model,
    prompts=[
        prompt_engineering(sport_session_name=sport_session, music_style="latino")
        for sport_session in ["yoga", "running", "weight training"]
    ],
    max_new_tokens=256,
    sampling_rate=32000,
    do_sample=True,
    guidance_scale=10
)

### Préparation de la prochaine séance

**Tâche 3:** Avec votre groupe, trouvez une musique (sans parole) associée à chacune des séance de sport suivante:
- yoga
- cardio training
- RPM/biking
- weight training (musculation)
- running (course à pied)

Les musiques doivent être au format `.mp3` et d'une durée de maximum 3min.
Le **groupe 1** aura pour thème la **musique classique**, et le **groupe 2 la musique electro**.

Vous trouverez dans la cellule ci-dessous des exemples de musiques associées au séances ci-dessus sur le thème **"musique: latine"**

In [ ]:
path_to_data = "./data/music/raw/latino"
music_files = [os.sep.join([path_to_data, f]) for f in os.listdir(path_to_data) if f[-4:] == ".mp3"]

for f in music_files:
    print(" ".join(f[:-4].split(os.sep)[-1].split("_") + [":"]))
    display(Audio(f, rate=44100))